## Final Report

## 1.Introduction
I have been learning IBM DataScience for a few weeks. Now I will try to analysis a real problem and create values.<br/>
My project is relevant to **sits selection** and it can be used as a recommendation system.My analysis was performed<br/>
in **Python**. The details are pushed to **Github**, containing detailed report and the Jupyter notebook. I will link them at<br/> the end of the report.


## 2.Business Problem
My friend Mike lived in **Chinatown** in **Manhattan** and he was prety ebjoied the venures in his neighborhood. <br/>
However, he has to move to Toronto recently for his new job.He wants to pick a new neighborhood which is similar <br/>
to the old one.The following criteria should be considered:
- The features of Chinatown in Manhattan.
- The features of Neighborhoods in Toronto.
- Determine how similar or dissimilar they are.


The assumption behind the analysis is that we can use unsupervised machine learning to create clusters of districts <br/>
that will provide us with a list of areas for potential locations for the new neighborhoods.

## 3.Data
To perform this analysis, we will need the following data:

- A dataset that contains the Manhattan as well as the the latitude and logitude coordinates of Chinatown.
- A neighborhoods data in Toronto.
- Venues's information of districts

Neighborhoods data in Manhattan will be obtained from dataset link: https://geo.nyu.edu/catalog/nyu_2451_34572<br/>
A neighborhoods data in Toronto will be obtained from wed:'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'<br/>
Top venues data will be obtained from Foursquare through an API.

In [5]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

***Download the data about neigorhoods in Manhattan***

In [6]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


***Set the data into pandas dataframe***

In [22]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
Neighbor_data=newyork_data['features']
columns_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']
Neighborhoods=pd.DataFrame(columns=columns_names)

In [25]:
for data in Neighbor_data:
    borough=data['properties']['borough']
    neighborhood_name=data['properties']['name']
    neighborhood_latlon=data['geometry']['coordinates']
    neighborhood_lat=neighborhood_latlon[1]
    neighborhood_lon=neighborhood_latlon[0]
    Neighborhoods=Neighborhoods.append({'Borough':borough,
                                        'Neighborhood':neighborhood_name,
                                        'Latitude':neighborhood_lat,
                                        'Longitude':neighborhood_lon},ignore_index=True)

In [26]:
Neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


***Get the data of Chinatown***

In [27]:
ChinaTown=Neighborhoods[(Neighborhoods['Neighborhood']=='Chinatown')]
ChinaTown

,Borough,Neighborhood,Latitude,Longitude
100,Manhattan,Chinatown,40.715618,-73.994279


***Download the dataset of Toronto***

In [28]:
page=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=pd.DataFrame(page[0])
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


***Clean the dataset***

In [52]:
df['Borough'].replace('Not assigned',np.nan,inplace=True)
df=df.dropna(subset=['Borough'],axis=0)
df=df[df.Borough.str.contains('Toronto',case=False)]
df.head()

,Postal Code,Borough,Neighborhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
13,M5B,Downtown Toronto,"Garden District, Ryerson"
22,M5C,Downtown Toronto,St. James Town
30,M4E,East Toronto,The Beaches


***Obtain LL***

In [53]:
df_LL=pd.read_csv('http://cocl.us/Geospatial_data')
df_all=pd.merge(df,df_LL)
df_all.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


***Pull the Mahantan data and the Toronto Data Together***

In [54]:
df_new=df_all[['Borough','Neighborhood','Latitude','Longitude']]
df_new.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,East Toronto,The Beaches,43.676357,-79.293031


In [55]:
row={'Borough':'Manhattan','Neighborhood':'Chinatown','Latitude':40.715618,'Longitude':-73.994279}
df_new=df_new.append(row,ignore_index=True)
df_new[(df_new['Neighborhood']=='Chinatown')]

,Borough,Neighborhood,Latitude,Longitude
39,Manhattan,Chinatown,40.715618,-73.994279


***Generate the final dataset***

In [56]:
CLIENT_ID = 'M4J1ULQCTBAARW1CHRL4LOSOU5OFG02MJRPQRNLTGRM02UP2' # your Foursquare ID
CLIENT_SECRET = 'B221FKTWRZIUAO5VLBO5HURLKAU42K0YIDXBQRJT4PXTBZ3W' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
Venues_all = getNearbyVenues(names=df_new['Neighborhood'],
                                   latitudes=df_new['Latitude'],
                                   longitudes=df_new['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [58]:
Venues_all.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


## 4.Methodology
In this part, we will direct our effort on detecting neighborhoods in Toronto whose venurs features are similar with Chinatow in Mahantan. <br/>
We will limit our analysis in **radius of 500 meters** and we only analysis the **top 100 venues**.<br/><br/>
In first step we have collected the required data: **location and type(category**) of every venue within 500 meters from the Neighborhood.<br/><br>
In second step we will cluster the neighborhoods using the **K-means** method. Then we will get several similar groups.<br/><br/>
In third and final step we will provide a list of neighborhoods which are similar with the Chinatown in Manhatan. We will also give a rank of the venues.

## 5.Analysis

***The frequence of venues***

In [60]:
onehot = pd.get_dummies(Venues_all['Venue Category'],prefix="", prefix_sep="")
onehot['Neighborhood'] = Venues_all['Neighborhood'] 
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]
onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
Grouped=onehot.groupby('Neighborhood').mean().reset_index()
Grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.017857,0.0,0.000000,0.0,0.000000,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.058824,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.066667,0.066667,0.133333,0.133333,0.133333,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
4,Central Bay Street,0.015152,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.015152,0.0,0.015152,0.0,0.015152,0.0,0.0


***Calculate the rank of veneus***

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Grouped['Neighborhood']

for ind in np.arange(Grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Seafood Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Climbing Gym,Burrito Place
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Pizza Place,Brewery,Burrito Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place


***Generate the recommondation list***

In [65]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [67]:
ks=6
Grouped_clustering=Grouped.drop('Neighborhood',1)
kmeans = KMeans(n_clusters=ks, random_state=0).fit(Grouped_clustering)
kmeans.labels_[0:10]
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Neighborhood_merged = Venues_all
Neighborhood_merged = Neighborhood_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Neighborhood_merged.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot


In [93]:
NeighborList=Neighborhood_merged[(Neighborhood_merged['Cluster Labels']==1)]

In [94]:
NeighborList=NeighborList[['Neighborhood','1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue','4th Most Common Venue','5th Most Common Venue']]

In [95]:
NeighborList=NeighborList.drop_duplicates()

In [96]:
NeighborList=NeighborList.reset_index()
NeighborList.drop(['index'],axis=1,inplace=True)

In [97]:
NeighborList

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Park,Bakery,Pub,Breakfast Spot
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,College Cafeteria,Beer Bar,Smoothie Shop
2,"Garden District, Ryerson",Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bubble Tea Shop
3,St. James Town,Café,Coffee Shop,Restaurant,Cocktail Bar,American Restaurant
4,The Beaches,Trail,Health Food Store,Pub,Department Store,Dessert Shop
5,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Seafood Restaurant
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place
7,Christie,Grocery Store,Café,Park,Candy Store,Diner
8,"Richmond, Adelaide, King",Coffee Shop,Café,Restaurant,Deli / Bodega,Hotel
9,"Dufferin, Dovercourt Village",Pharmacy,Bakery,Coffee Shop,Bank,Middle Eastern Restaurant


***This list includs all the neighborhoods which are similar with the Chinatown. Besides we can see the top five venues in each neighborhood.<br/>
Based on this list, Mike could select the best place to live.***

## 6.Result and Discussion
<br/>
Our analysis shows that there are 33 neighborhoods are similar with the Chinatown which means Mike has many different choices.<br/>
Although the meighborhoods in the table above are similar, the ranks of venues are various. If Mike want to select the best place,<br/>
he can conciderate each verious carfully. For instance, if the Chinese Restaurant is the most indispensiable venue. The St. James Town <br/>
or the Cabbagetown maybe a good choice.


## 7.Conclusion
Purpose of this project was to identify neighborhoods in Toronto whose revues features are similar with the Chinatown in Mahantan.<br/>
By calculating the dataset. We collect various location data and rebuild them, then cluster 6 groups by using k_means method. In order <br/>
to give more details, we calculate the ranks of venues in each neighborhood. arting points for final exploration by stakeholders.<br/><br/>
Final decission on optimal neighborhood location will be made by Mike based on specific characteristics of neighborhoods , taking into <br/>
consideration additional factors like frequence of each venue.